In [12]:
from gurobipy import Model, GRB, quicksum
import pandas as df


In [2]:
m = Model("Optimization")


Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [13]:
data = df.read_csv('/Users/aimaldastagirzada/Downloads/price_response (1).csv')

In [22]:
from gurobipy import Model, GRB
import gurobipy as gb

# Assume the dataset is loaded into a variable named 'data'
# Cross-elasticity factor
cross_elasticity = 0.1

# Create a new model
model = Model("TechEssentials Pricing Optimization with Capacity")

# Decision variables: Price for each of the 9 products
prices = model.addVars(len(data), lb=0, vtype=GRB.CONTINUOUS, name="Price")

# Demand variables for each product, adjusted for cross-elasticity and data-driven
demands = model.addVars(len(data), lb=0, vtype=GRB.CONTINUOUS, name="Demand")

# Update demand calculations to include data-driven parameters and cross-elasticity
for i in range(len(data)):
    intercept, sensitivity = data.loc[i, ['Intercept', 'Sensitivity']]
    # Base demand from price, adjusted for intercept and sensitivity from the dataset
    demand_expr = intercept + sensitivity * prices[i]
    
    # Add cross-elasticity effects from other products within the same line
    for j in range((i // 3) * 3, (i // 3) * 3 + 3):
        if i != j:
            demand_expr += cross_elasticity * (prices[j] - prices[i])
    
    model.addConstr(demands[i] == demand_expr, name=f"DemandCalc_{i}")

# Capacity constraints
for i in range(len(data)):
    capacity = data.loc[i, 'Capacity']
    model.addConstr(demands[i] <= capacity, name=f"Capacity_{i}")

# Objective: Maximize total revenue
model.setObjective(gb.quicksum(prices[i] * demands[i] for i in range(len(data))), GRB.MAXIMIZE)

# Optimize the model
model.optimize()

# Display results (Pseudo-code; replace with actual Gurobi syntax if running in a compatible environment)
if model.status == GRB.OPTIMAL:
    print("Optimal Total Revenue: ${:.2f}".format(model.ObjVal))
    print("Optimal Prices and Expected Demands:")
    for i in range(len(data)):
        print(f"{data.loc[i, 'Product']}: Price = ${prices[i].X:.2f}, Demand = {demands[i].X:.0f}")
else:
    print("Optimal solution was not found.")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D56)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 18 rows, 18 columns and 45 nonzeros
Model fingerprint: 0x7edd169a
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-01, 5e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 9e+04]
Presolve removed 9 rows and 0 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 15 rows and 12 columns
Presolve time: 0.12s
Presolved: 10 rows, 11 columns, 28 nonzeros
Presolved model has 3 bilinear constraint(s)
         in product terms.
         Presolve was not able to compute smaller bounds for these variables.
         Consider bounding these variables or reformulating the model.

Variable types: 11 continuous, 0 integer (0 binary)
Found heuristic solution: ob

In [15]:
print(model)

<gurobi.Model MIP instance TechEssentials Pricing Optimization with Capacity: 18 constrs, 18 vars, Parameter changes: Username=(user-defined)>


In [24]:
import gurobipy as gp
from gurobipy import GRB

try:
    # Create a new model
    m = gp.Model("TechEssentials_Optimization")

    # Create variables
    p1 = m.addVar(name="p1", lb=0)  # Price of Basic version
    p2 = m.addVar(name="p2", lb=0)  # Price of Advanced version

    # Coefficients for the linear price response function
    # Replace these values with the actual coefficients
    a1, a2, b1, b2 = 1, 1, 1, 1  # Example coefficients

    # Set the objective: Maximize revenue
    m.setObjective(p1 * (a1 - b1 * p1) + p2 * (a2 - b2 * p2), GRB.MAXIMIZE)

    # Add constraint: Advanced version should have a higher price than the Basic one
    m.addConstr(p2 >= p1, "Price_Ordering")

    # Optimize model
    m.optimize()

    # Print optimal values
    if m.status == GRB.OPTIMAL:
        print(f"Optimal Price of Basic Version (p1): {p1.X}")
        print(f"Optimal Price of Advanced Version (p2): {p2.X}")
    else:
        print("Optimal solution was not found.")

except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    print("Encountered an attribute error.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D56)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0x882e4b4d
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.05s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -4.99000000e+05  5.00000000e+05  0.00e+00 0.00e+00  1.00e+06     0s
   1  -1.95652945e+05  1.